In [21]:
%matplotlib notebook
import logging
import os
import math
import multiprocessing
import json
import re
from itertools import combinations 

from scipy.stats import t
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

## Grant Breaking impacts

In [22]:
threeGPP_low    = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/nonPeriodic-3GPP-006vpm-normal-grant-2020-07-30-19_53_43"
threeGPP_medium = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/nonPeriodic-3GPP-012vpm-normal-grant-2020-07-30-19_53_43"
threeGPP_high   = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/nonPeriodic-3GPP-02vpm-normal-grant-2020-07-30-19_53_43"

etsi_low      = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/nonPeriodic-ETSI-006vpm-normal-grant-2020-07-31-09_34_33"
etsi_medium   = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/nonPeriodic-ETSI-012vpm-490-2020-07-30-15_50_44"
etsi_high     = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/nonPeriodic-ETSI-02vpm-normal-grant-2020-07-31-09_34_33"

one_shot_low    = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/reschedule-006vpm-2020-07-31-09_37_50"
one_shot_medium = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/reschedule-012vpm-2020-07-31-09_37_50"
one_shot_high   = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/reschedule-02vpm-2020-07-31-10_42_40"

periodic_low    = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/periodic-006vpm-2020-07-30-19_55_34"
periodic_medium = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/periodic-012vpm-2020-07-30-19_55_34"
periodic_high   = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x/periodic-02vpm-2020-07-30-19_55_34"


In [26]:
densities = { 
    "low": {
        "3GPP": threeGPP_low,
        "ETSI": etsi_low,
        "Periodic": periodic_low,
        "One-Shot": one_shot_low
    },
    "medium": {
        "3GPP": threeGPP_medium,
        "ETSI": etsi_medium,
        "Periodic": periodic_medium,
        "One-Shot": one_shot_medium
    },
    "high": {
        "3GPP": threeGPP_high,
        "ETSI": etsi_high,
        "Periodic": periodic_high,
        "One-Shot": one_shot_high
    }
}

In [27]:
for density in densities:
    if density == "medium":
        continue
    print("="*26)
    print("Density: {}".format(density))
    collisions_dfs = []

    for config in densities[density]:
        print("="*26)
        print("Config: {}".format(config))
        df = pd.read_csv("{}/run-2.csv".format(densities[density][config]))

        position_df = df[["Time", "NodeID", "posX", "posY"]]
        position_df = position_df.dropna()

        grant_df = df[["NodeID", "Time", "grantBreak", "grantBreakMissedTrans", 
                       "grantBreakSize", "grantBreakTiming","grantStartTime", "selectedSubchannelIndex",
                       "resourceReselectionCounter", "posX", "posY"]]

        grant_df = grant_df.dropna(subset=["grantBreak", "grantBreakMissedTrans", "grantBreakSize",
                                           "grantBreakTiming", "grantStartTime", "selectedSubchannelIndex",
                                           "resourceReselectionCounter"], how="all")

        grant_description = grant_df[grant_df["grantStartTime"].notnull()]
        grant_description = grant_description[["NodeID", "grantStartTime", 
                                               "selectedSubchannelIndex", 
                                               "resourceReselectionCounter"]].reset_index(drop=True)

        grant_breaking_df = grant_df[["NodeID", "Time", "grantBreak", "grantBreakMissedTrans", "grantBreakSize", "grantBreakTiming"]]
        grant_breaking_df = grant_breaking_df.dropna(subset=["grantBreak", "grantBreakMissedTrans", "grantBreakSize", "grantBreakTiming"], how="all")
        grant_breaking_df["earlyGrantBreak"] = np.where(
            (grant_breaking_df["grantBreakMissedTrans"].eq(1)) | 
            (grant_breaking_df["grantBreakSize"].eq(1)) | 
            (grant_breaking_df["grantBreakTiming"].eq(1)), 1, 0)
        grant_breaking_df["grantBreak"] = np.where(grant_breaking_df["earlyGrantBreak"].eq(1), 0, 1)
        grant_breaking_df = grant_breaking_df[["NodeID", "Time", "grantBreak", "earlyGrantBreak"]]
        grant_description = grant_description.reset_index(drop=True)

        combined_df = pd.merge(grant_description, grant_breaking_df, left_on=["NodeID"], right_on=["NodeID"], how='right')
        combined_df = combined_df[(combined_df["grantStartTime"] < combined_df["Time"])]
        combined_df = combined_df.drop_duplicates(subset=["NodeID", "grantStartTime"], keep="first")
        combined_df = combined_df.reset_index(drop=True)
        combined_df = combined_df.sort_values(by=["grantStartTime"]).reset_index(drop=True)
        combined_df = combined_df.rename(columns={"Time": "grantBreakTime"})

        if config == "One-Shot":
            combined_df["grantTransmissions"] = (
                (combined_df["grantBreakTime"] - combined_df["grantStartTime"]) / 0.1)
            combined_df["grantTransmissions"] = np.ceil(combined_df["grantTransmissions"])
            combined_df.loc[combined_df.grantTransmissions == 2, 'grantTransmissions'] = 1
        else:
            combined_df["timeDiff"] = combined_df["grantBreakTime"] - combined_df["grantStartTime"]
            combined_df["trans2"] = combined_df["timeDiff"] / 0.1
            combined_df["trans3"] = combined_df["trans2"].round()
            combined_df["grantTransmissions"] = combined_df["trans3"].astype(int)

        subchannel_number = grant_description["selectedSubchannelIndex"].max() + 1
        min_time = grant_description["grantStartTime"].min()
        max_time = grant_description["grantStartTime"].max()

        all_possible_times = np.arange(min_time, max_time, 0.001)
        num_subchannels = len(all_possible_times) * subchannel_number

        combined_df["averageGrantUsage"] = (combined_df["grantTransmissions"] / combined_df["resourceReselectionCounter"]) * 100
        combined_df["averageGrantUsage"] = combined_df["averageGrantUsage"].astype(int)

        average_grant_transmissions = combined_df["grantTransmissions"].mean()
        average_grant_length = combined_df["resourceReselectionCounter"].mean()
        early_breaks = combined_df["earlyGrantBreak"].sum()
        total_grants = combined_df["grantStartTime"].count()

        if config == "One-Shot":
            sending_df = combined_df[["grantStartTime", "selectedSubchannelIndex"]]
            broken_df = combined_df[["grantStartTime", "selectedSubchannelIndex"]]
            broken_df["UnusedTime"] = broken_df["grantStartTime"] + 0.1
            broken_df = broken_df[["UnusedTime", "selectedSubchannelIndex"]]

            sending_df["Time"] = round(sending_df["grantStartTime"], 3)
            broken_df["Time"] = round(broken_df["UnusedTime"], 3)

            sending_df = sending_df[["Time", "selectedSubchannelIndex"]]
            broken_df = broken_df[["Time", "selectedSubchannelIndex"]]

            together = pd.merge(sending_df, broken_df, on=["Time", "selectedSubchannelIndex"], how="inner")
            together["Time"] = round(together["Time"], 3)

            together.reset_index(inplace=True, drop=True)
            sending_df.reset_index(inplace=True, drop=True)
            broken_df.reset_index(inplace=True, drop=True)

            sending_df = sending_df.loc[pd.merge(sending_df, together, on=['Time','selectedSubchannelIndex'], how='left', indicator=True)['_merge'] == 'left_only']
            broken_df = broken_df.loc[pd.merge(broken_df, together, on=['Time','selectedSubchannelIndex'], how='left', indicator=True)['_merge'] == 'left_only']

            together_collisions = together.pivot_table(index=['Time','selectedSubchannelIndex'], aggfunc='size')
            count_sends = sending_df.pivot_table(index=['Time','selectedSubchannelIndex'], aggfunc='size')
            count_missed = broken_df.pivot_table(index=['Time','selectedSubchannelIndex'], aggfunc='size')

            duplicates_together = together_collisions[together_collisions > 1]
            duplicates_sends = count_sends[count_sends > 1]
            duplicates_missed = count_missed[count_missed > 1]
            
            reserved_occupied_count = duplicates_together.count()
            occupied_occupied_count = duplicates_sends.count()
            reserved_reserved_count = duplicates_missed.count()

            together = together.drop_duplicates()
            sending_df = sending_df.drop_duplicates()
            broken_df = broken_df.drop_duplicates()

            booked_free = broken_df.shape[0] + together.shape[0]

            total = len(np.arange(500, 512, 0.001)) * 3

            correct_use = sending_df.shape[0]

            free = total - correct_use - booked_free

        else:

            sending_df = df[["Time", "NodeID", "tbSent", "sciSent", "subchannelSent"]]

            sending_df = sending_df.dropna(subset=["sciSent"])
            sending_df = sending_df.fillna(0)

            missed_trans = sending_df[sending_df["tbSent"] == 0]

            correct_trans = sending_df[sending_df["tbSent"] == 1]
            correct_trans = correct_trans[["Time", "subchannelSent"]]
            correct_trans["Time"] = round(correct_trans["Time"], 3)

            missed_trans = missed_trans[["Time", "subchannelSent"]]
            missed_trans["Time"] = round(missed_trans["Time"], 3)

            together = pd.merge(missed_trans, correct_trans, on=["Time", "subchannelSent"], how="inner")
            together["Time"] = round(together["Time"], 3)

            together.reset_index(inplace=True, drop=True)
            correct_trans.reset_index(inplace=True, drop=True)
            missed_trans.reset_index(inplace=True, drop=True)

            correct_trans = correct_trans.loc[pd.merge(correct_trans, together, on=['Time','subchannelSent'], how='left', indicator=True)['_merge'] == 'left_only']
            missed_trans = missed_trans.loc[pd.merge(missed_trans, together, on=['Time','subchannelSent'], how='left', indicator=True)['_merge'] == 'left_only']

            together_collisions = together.pivot_table(index=['Time','subchannelSent'], aggfunc='size')
            count_sends = correct_trans.pivot_table(index=['Time','subchannelSent'], aggfunc='size')
            count_missed = missed_trans.pivot_table(index=['Time','subchannelSent'], aggfunc='size')

            duplicates_together = together_collisions[together_collisions > 1]
            duplicates_sends = count_sends[count_sends > 1]
            duplicates_missed = count_missed[count_missed > 1]
            
            reserved_occupied_count = duplicates_together.count()
            occupied_occupied_count = duplicates_sends.count()
            reserved_reserved_count = duplicates_missed.count()

            together = together.drop_duplicates()
            correct_trans = correct_trans.drop_duplicates()
            missed_trans = missed_trans.drop_duplicates()

            booked_free = missed_trans.shape[0] + together.shape[0]

            correct_use = correct_trans.shape[0]

            total = len(np.arange(500, 512, 0.001)) * 3

            free = total - correct_use - booked_free

        combined_df = pd.merge(combined_df, position_df, left_on=["NodeID", "grantStartTime"], right_on=["NodeID", "Time"])
        combined_df = combined_df.drop(columns=["Time"])
        messing_df = combined_df
        messing_df = messing_df.round({'grantStartTime': 3, 'grantBreakTime': 3})
        messing_df['selectedSubframe'] = messing_df['grantStartTime'].apply(lambda x: (round(x, 3) - math.floor(x)) * 1000)
        messing_df["hundreds"] = messing_df["selectedSubframe"] // 100
        messing_df["selectedSubframe"] = messing_df["selectedSubframe"] - messing_df["hundreds"] * 100
        messing_df = messing_df.drop(columns=["hundreds"])
        messing_df = messing_df[["NodeID", "grantStartTime", "grantBreakTime", "selectedSubframe", "selectedSubchannelIndex", "grantBreak", "earlyGrantBreak", "grantTransmissions", "resourceReselectionCounter", "posX", "posY"]]
        new_df = pd.merge(messing_df, messing_df, on=["selectedSubframe", "selectedSubchannelIndex"])
        new_df = new_df[new_df["NodeID_x"] != new_df["NodeID_y"]]
        new_df = new_df[(new_df["grantStartTime_x"] <= new_df["grantStartTime_y"]) & (new_df["grantBreakTime_x"] >= new_df["grantStartTime_y"])]
        new_df = new_df[['NodeID_x', 'grantStartTime_x', 'grantBreakTime_x', 'selectedSubframe',
                         'selectedSubchannelIndex', 'NodeID_y','grantStartTime_y', 'grantBreakTime_y',
                         'posX_x', 'posY_x', 'posX_y', 'posY_y']]
        new_df["Collision_distance"] = np.sqrt(
                    np.square(new_df["posX_x"] - new_df["posX_y"]) +
                    np.square(new_df["posY_x"] - new_df["posY_y"]))
        new_df["Collisions"] = (new_df[['grantBreakTime_x', 'grantBreakTime_y']].min(axis=1) - new_df[["grantStartTime_x", "grantStartTime_y"]].max(axis=1)) * 10
        new_df = new_df[["NodeID_x", "NodeID_y", "selectedSubframe", "selectedSubchannelIndex", "Collisions", "Collision_distance"]]
        new_df = new_df.iloc[::2, :]
        new_df.reset_index(inplace=True, drop=True)
        max_distance = min(530, new_df["Collision_distance"].max())
        distance_df = new_df.groupby(pd.cut(new_df["Collision_distance"], np.arange(0, max_distance, 25))).agg("sum")

        colliding_transmissions = new_df["Collisions"].sum()

        print("AverageGrantLength: {}, grantLength STD: {}".format(average_grant_length, combined_df["resourceReselectionCounter"].std()))
        print("Average grant Transmissions: {}, grantTransmissions STD: {}".format(average_grant_transmissions, combined_df["grantTransmissions"].std()))
              
        print("-" * 16)
        print("Resources based on used resources")
        print("Percentage Used: {}".format(((correct_use + booked_free)/ total) * 100))
        print("Resources Occupied + Reserved Unused: {}%".format((reserved_occupied_count / (correct_use + booked_free)) * 100))
        print("Resources Occupied by 2+: {}%".format((occupied_occupied_count / (correct_use + booked_free)) * 100))
        print("Resources Reserved Unused by 2+: {}%".format((reserved_reserved_count / (correct_use + booked_free)) * 100))
              
        reserved_occupied_count = duplicates_together.count()
        occupied_occupied_count = duplicates_sends.count()
        reserved_reserved_count = duplicates_missed.count()
        
        print((average_grant_transmissions/average_grant_length) * 100)
        print(combined_df["averageGrantUsage"].mean())

        if config == "ETSI":            
            used_etsi = (correct_use/total)*100
            unused_etsi = (free/total)*100
            booked_but_free_etsi = (booked_free/total) * 100
            grants_used_wrongly_etsi = (early_breaks / total_grants) * 100
            average_grant_usage_etsi = (average_grant_transmissions/average_grant_length) * 100

            grant_ci_etsi = 1.96 * combined_df["averageGrantUsage"].std()

            collisions_dfs.append(distance_df)
        elif config == "3GPP":
            used_3gpp = (correct_use/total)*100
            unused_3gpp = (free/total)*100
            booked_but_free_3gpp = (booked_free/total) * 100
            grants_used_wrongly_3gpp = (early_breaks / total_grants) * 100
            average_grant_usage_3gpp = (average_grant_transmissions/average_grant_length) * 100
            grant_ci_3gpp =  1.96 * combined_df["averageGrantUsage"].std()

            collisions_dfs.append(distance_df)
        elif config == "Periodic":
            used_periodic = (correct_use/total)*100
            unused_periodic = (free/total)*100
            booked_but_free_periodic = (booked_free/total) * 100
            grants_used_wrongly_periodic = (early_breaks / total_grants) * 100
            average_grant_usage_periodic = (average_grant_transmissions/average_grant_length) * 100
            grant_ci_periodic = 1.96 * combined_df["averageGrantUsage"].std()

            collisions_dfs.append(distance_df)
        elif config == "One-Shot":
            used_one_shot = (correct_use/total)*100
            unused_one_shot = (free/total)*100
            booked_but_free_one_shot = (booked_free/total) * 100
            grants_used_wrongly_one_shot = (early_breaks / total_grants) * 100
            average_grant_usage_one_shot = (average_grant_transmissions/average_grant_length) * 100
            grant_ci_one_shot = 1.96 * combined_df["averageGrantUsage"].std()

            collisions_dfs.append(distance_df)

    plt.ioff()

    fig, ax = plt.subplots()

    labels = ['Aperiodic\n(3GPP)', 'Aperiodic\n(ETSI)', 'Periodic\n(Default)', 'Periodic\n(Single Slot Usage)']
    booked_but_free = [booked_but_free_3gpp, booked_but_free_etsi, booked_but_free_periodic, booked_but_free_one_shot]
    used = [used_3gpp, used_etsi, used_periodic, used_one_shot]
    unused = [unused_3gpp, unused_etsi, unused_periodic, unused_one_shot]

    width = .2  # the width of the bars

    r1 = np.arange(len(labels))
    r2 = [x + width for x in r1]
    r3 = [x + width for x in r2]

    patterns = ['-', '+', 'x', '\\', '*', 'o', 'O', '.']
    rects1 = ax.bar(r1, booked_but_free, width, label='Reserved but unused', color="blue", lw=1., zorder = 0)
    rects2 = ax.bar(r2, used, width, label='Occupied', color='none', edgecolor='green', hatch='//', lw=1., zorder = 0)
    rects3 = ax.bar(r3, unused, width, label='Free', align="center", edgecolor='orange', fill=None, lw=1., zorder = 0)

    ax.set_ylabel('Resources Occupancy %')
    plt.xticks([r + width for r in range(len(labels))], labels)
    ax.set_xticklabels(labels)
    ax.legend(prop={'size': 10})
    ax.set_ylim(0, 100)
    plt.yticks(np.arange(0, 101, 10))

    fig.tight_layout()

    plt.savefig("../data/figures/{}-resources.png".format(density), dpi=400)

    plt.close(fig)


    fig, ax = plt.subplots()

    labels = ['Aperiodic\n(3GPP)', 'Aperiodic\n(ETSI)', 'Periodic\n(Default)', 'Periodic\n(Single Slot Usage)']
    percentages = [grants_used_wrongly_3gpp, grants_used_wrongly_etsi,
                   grants_used_wrongly_periodic, grants_used_wrongly_one_shot]

    width = 0.3  # the width of the bars

    r1 = np.arange(len(labels))

    rects1 = ax.bar(r1, percentages, width)

    ax.set_ylabel('Total Grants Breaks (%) ')
    ax.set_ylim(0, 100)
    plt.yticks(np.arange(0, 101, 10))
    plt.xticks([r for r in range(len(labels))], labels)
    ax.set_xticklabels(labels)

    fig.tight_layout()

    plt.savefig("../data/figures/{}-grants.png".format(density), dpi=400)

    plt.close(fig)


    fig, ax = plt.subplots()

    labels = ['Aperiodic\n(3GPP)', 'Aperiodic\n(ETSI)', 'Periodic\n(Default)', 'Periodic\n(Single Slot Usage)']
    percentages = [average_grant_usage_3gpp, average_grant_usage_etsi,
                   average_grant_usage_periodic, average_grant_usage_one_shot]
    yerror = [grant_ci_3gpp, grant_ci_etsi, grant_ci_periodic, grant_ci_one_shot]

    width = 0.3  # the width of the bars

    r1 = np.arange(len(labels))

    rects1 = ax.bar(r1, percentages, width, yerr=yerror, capsize=10)

    ax.set_ylabel('Average grant usage (%) ')
    ax.set_ylim(0, 100)
    plt.yticks(np.arange(0, 101, 10))
    plt.xticks([r for r in range(len(labels))], labels)
    ax.set_xticklabels(labels)

    fig.tight_layout()

    plt.savefig("../data/figures/{}-Grant-Usage.png".format(density), dpi=400)

    plt.close(fig)


    fig, ax = plt.subplots()

    colours = ["black", "red", "blue", "green"]

    labels = ['Aperiodic\n(3GPP)', 'Aperiodic\n(ETSI)', 'Periodic\n(Default)', 'Periodic\n(Single Slot Usage)']

    distances = np.arange(0, 520, 25)

    for label, df, colour in zip(labels, collisions_dfs, colours):
        ax.plot(distances, df["Collisions"], label=label, fillstyle="none", marker="x", markevery=2 ,color=colour, linestyle="solid")

    ax.set(xlabel='Distance (m)', ylabel="Collision Count")
    ax.legend(loc="lower left")
    ax.tick_params(direction='in')

    ax.set_xlim([0, 500])

    plt.xticks(np.arange(0, (max(distances) + 1), step=50))
    plt.grid(b=True, alpha=0.5)

    plt.savefig("../data/figures/{}-collisions.png".format(density), dpi=400)

    plt.close(fig)


Density: low
Config: 3GPP


EmptyDataError: No columns to parse from file

In [28]:
df = pd.read_csv("{}/run-2.csv".format(densities["medium"]["3GPP"]))

position_df = df[["Time", "NodeID", "posX", "posY"]]
position_df = position_df.dropna()

grant_df = df[["NodeID", "Time", "grantBreak", "grantBreakMissedTrans", 
               "grantBreakSize", "grantBreakTiming","grantStartTime", "selectedSubchannelIndex",
               "resourceReselectionCounter", "posX", "posY"]]

grant_df = grant_df.dropna(subset=["grantBreak", "grantBreakMissedTrans", "grantBreakSize",
                                   "grantBreakTiming", "grantStartTime", "selectedSubchannelIndex",
                                   "resourceReselectionCounter"], how="all")

grant_description = grant_df[grant_df["grantStartTime"].notnull()]
grant_description = grant_description[["NodeID", "grantStartTime", 
                                       "selectedSubchannelIndex", 
                                       "resourceReselectionCounter"]].reset_index(drop=True)

grant_breaking_df = grant_df[["NodeID", "Time", "grantBreak", "grantBreakMissedTrans", "grantBreakSize", "grantBreakTiming"]]
grant_breaking_df = grant_breaking_df.dropna(subset=["grantBreak", "grantBreakMissedTrans", "grantBreakSize", "grantBreakTiming"], how="all")
grant_breaking_df["earlyGrantBreak"] = np.where(
    (grant_breaking_df["grantBreakMissedTrans"].eq(1)) | 
    (grant_breaking_df["grantBreakSize"].eq(1)) | 
    (grant_breaking_df["grantBreakTiming"].eq(1)), 1, 0)
grant_breaking_df["grantBreak"] = np.where(grant_breaking_df["earlyGrantBreak"].eq(1), 0, 1)
grant_breaking_df = grant_breaking_df[["NodeID", "Time", "grantBreak", "earlyGrantBreak"]]
grant_description = grant_description.reset_index(drop=True)

combined_df = pd.merge(grant_description, grant_breaking_df, left_on=["NodeID"], right_on=["NodeID"], how='right')
combined_df = combined_df[(combined_df["grantStartTime"] < combined_df["Time"])]
combined_df = combined_df.drop_duplicates(subset=["NodeID", "grantStartTime"], keep="first")
combined_df = combined_df.reset_index(drop=True)
combined_df = combined_df.sort_values(by=["grantStartTime"]).reset_index(drop=True)
combined_df = combined_df.rename(columns={"Time": "grantBreakTime"})

combined_df["grantTransmissions"] = ((combined_df["grantBreakTime"] - combined_df["grantStartTime"]) / 0.1)

subchannel_number = grant_description["selectedSubchannelIndex"].max() + 1
min_time = grant_description["grantStartTime"].min()
max_time = grant_description["grantStartTime"].max()

combined_df["averageGrantUsage"] = combined_df["grantTransmissions"] / combined_df["resourceReselectionCounter"]

average_grant_transmissions = combined_df["grantTransmissions"].mean()
average_grant_length = combined_df["resourceReselectionCounter"].mean()
early_breaks = combined_df["earlyGrantBreak"].sum()
total_grants = combined_df["grantStartTime"].count()

In [68]:
df = pd.read_csv("/Users/brianmccarthy/git_repos/results-analysis/data/processed_data/cv2x/nonPeriodic-ETSI-012vpm-normal-grant-2020-07-27-09_53_31/CBR.csv")

In [69]:
df.head()

,Mean,Time,Confidence-Interval
0,NaN,199.0,NaN
1,0.167971,199.1,0.057088
2,0.239758,199.2,0.049132
3,0.138946,199.3,0.031477
4,0.153564,199.4,0.041174


In [70]:
df["Time"] = df["Time"] - df["Time"].min()

In [71]:
df.head()

,Mean,Time,Confidence-Interval
0,NaN,0.0,NaN
1,0.167971,0.1,0.057088
2,0.239758,0.2,0.049132
3,0.138946,0.3,0.031477
4,0.153564,0.4,0.041174


In [ ]:
etsi_low
etsi_medium
etsi_high

In [29]:
etsi_low_df    = pd.read_csv("{}/run-1.csv".format(etsi_low))
etsi_medium_df = pd.read_csv("{}/run-1.csv".format(etsi_medium))
etsi_high_df   = pd.read_csv("{}/run-1.csv".format(etsi_high))

In [68]:
print("====== Vehicle count ======")
print("Low: {}".format(etsi_low_df["NodeID"].max()))
print("Medium: {}".format(etsi_medium_df["NodeID"].max()))
print("High: {}".format(etsi_high_df["NodeID"].max()))

print("======= Trans Num =========")
print("Low: {}".format(etsi_low_df["tbSent"].count()))
print("Medium: {}".format(etsi_medium_df["tbSent"].count()))
print("High: {}".format(etsi_high_df["tbSent"].count()))

print("====== Trans per vehicle ======")
print("Low: {}".format(etsi_low_df["tbSent"].count()/etsi_low_df["NodeID"].max()))
print("Medium: {}".format(etsi_medium_df["tbSent"].count()/etsi_medium_df["NodeID"].max()))
print("High: {}".format(etsi_high_df["tbSent"].count()/etsi_high_df["NodeID"].max()))

print("======= Trans Number ========")
print("Low: {}".format((etsi_low_df["tbSent"].count()/etsi_low_df["NodeID"].max())/12))
print("Medium: {}".format((etsi_medium_df["tbSent"].count()/etsi_medium_df["NodeID"].max())/12))
print("High: {}".format((etsi_high_df["tbSent"].count()/etsi_high_df["NodeID"].max())/12))

print("======= Trans Rate ========")
print("Low: {}".format(1000/((etsi_low_df["tbSent"].count()/etsi_low_df["NodeID"].max())/12)))
print("Medium: {}".format(1000/((etsi_medium_df["tbSent"].count()/etsi_medium_df["NodeID"].max())/12)))
print("High: {}".format(1000/((etsi_high_df["tbSent"].count()/etsi_high_df["NodeID"].max())/12)))

====== Vehicle count ======
Low: 125
Medium: 245
High: 401
======= Trans Num =========
Low: 10117
Medium: 11425
High: 11910
====== Trans per vehicle ======
Low: 80.936
Medium: 46.63265306122449
High: 29.700748129675812
======= Trans Number ========
Low: 6.744666666666667
Medium: 3.886054421768707
High: 2.475062344139651
======= Trans Rate ========
Low: 148.2652960363744
Medium: 257.3304157549234
High: 404.0302267002519


In [62]:
etsi_low_df[(etsi_low_df["NodeID"] == 100) & (etsi_low_df["tbSent"] > 0)]

,NodeID,Time,cbr,generatedGrants,grantBreak,grantBreakMissedTrans,grantBreakSize,grantBreakTiming,grantStartTime,interPacketDelay,...,tbDecoded,tbFailedButSCIReceived,tbFailedDueToInterference,tbFailedDueToNoSCI,tbFailedDueToProp,tbFailedHalfDuplex,tbReceived,tbSent,txRxDistanceSCI,txRxDistanceTB
14521,100,500.071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
37527,100,500.182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1443.279421,1443.279421
76973,100,500.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
114559,100,500.557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,683.155760,683.155760
135138,100,500.657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,680.132890,680.132890
156312,100,500.757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,673.543289,-1.000000
176410,100,500.857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
196652,100,500.957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
222812,100,501.090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
264471,100,501.301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [63]:
etsi_medium_df[(etsi_medium_df["NodeID"] == 100) & (etsi_medium_df["tbSent"] > 0)]

,NodeID,Time,cbr,generatedGrants,grantBreak,grantBreakMissedTrans,grantBreakSize,grantBreakTiming,grantStartTime,interPacketDelay,...,tbDecoded,tbFailedButSCIReceived,tbFailedDueToInterference,tbFailedDueToNoSCI,tbFailedDueToProp,tbFailedHalfDuplex,tbReceived,tbSent,txRxDistanceSCI,txRxDistanceTB
31109,100,500.063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1026.129746,1026.129746
145619,100,500.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1069.873749,1069.873749
292384,100,500.604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
376299,100,500.778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,972.415413,-1.000000
507475,100,501.044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,777.302988,1141.050874
621817,100,501.280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,459.795371,-1.000000
768153,100,501.581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1230.156895,326.939391
872996,100,501.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,247.437488,-1.000000
981810,100,502.033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,679.549161,-1.000000
1104683,100,502.286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,516.607895,-1.000000


In [64]:
etsi_high_df[(etsi_high_df["NodeID"] == 100) & (etsi_high_df["tbSent"] > 0)]

,NodeID,Time,cbr,generatedGrants,grantBreak,grantBreakMissedTrans,grantBreakSize,grantBreakTiming,grantStartTime,interPacketDelay,...,tbDecoded,tbFailedButSCIReceived,tbFailedDueToInterference,tbFailedDueToNoSCI,tbFailedDueToProp,tbFailedHalfDuplex,tbReceived,tbSent,txRxDistanceSCI,txRxDistanceTB
107030,100,500.123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,110.702604,-1.000000
419487,100,500.507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1299.033313,772.269090
786918,100,500.954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1233968,100,501.479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1064.059517,-1.000000
1570825,100,501.879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1406.080168,798.487761
1926196,100,502.303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,723.168728,723.168728
2333875,100,502.804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,874.615941,-1.000000
2656194,100,503.198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,514.769172,-1.000000
3065447,100,503.703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,443.997321,-1.000000
3399610,100,504.113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1446.992009,-1.000000
